## Ensembling

In [1]:
from lightning import pytorch as pl
import numpy as np
import torch
from chemprop import data, models, nn

This is an example [dataloader](./data/dataloaders.ipynb).

In [2]:
smis = ["C" * i for i in range(1, 4)]
ys = np.random.rand(len(smis), 1)
dset = data.MoleculeDataset([data.MoleculeDatapoint.from_smi(smi, y) for smi, y in zip(smis, ys)])
dataloader = data.build_dataloader(dset)

### Model ensembling

A single model will sometimes give erroneous predictions for some molecules. These erroneous predictions can be mitigated by averaging the predictions of several models trained on the same data. 

In [3]:
ensemble = []
n_models = 3
for _ in range(n_models):
    ensemble.append(models.MPNN(nn.BondMessagePassing(), nn.MeanAggregation(), nn.RegressionFFN()))

In [4]:
for model in ensemble:
    trainer = pl.Trainer(logger=False, enable_checkpointing=False, max_epochs=1)
    trainer.fit(model, dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/knathan/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
Loading `train_dataloader` to estimate number of stepping batches.
/home/knathan/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.

  | Name            | Type               | Params
-------------------------------------------------------
0 | message_passing | BondMessagePassing | 227 K 
1 | agg             | MeanAggregation    | 0     
2 | bn              | BatchNorm1d        | 60

Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  2.92it/s, train_loss=0.466]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  2.90it/s, train_loss=0.466]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


...

Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  1.59it/s, train_loss=0.468]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Loading `train_dataloader` to estimate number of stepping batches.

  | Name            | Type               | Params
-------------------------------------------------------
0 | message_passing | BondMessagePassing | 227 K 
1 | agg             | MeanAggregation    | 0     
2 | bn              | BatchNorm1d        | 600   
3 | predictor       | RegressionFFN      | 90.6 K
4 | X_d_transform   | Identity           | 0     
-------------------------------------------------------
318 K     Trainable params
0         Non-trainable params
318 K     Total params
1.276     Total estimated model params size (MB)



Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  1.35it/s, train_loss=0.398]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  1.34it/s, train_loss=0.398]


In [5]:
prediction_dataloader = data.build_dataloader(dset, shuffle=False)
predictions = []
for model in ensemble:
    predictions.append(torch.concat(trainer.predict(model, prediction_dataloader)))

/home/knathan/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Predicting DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]

Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 29.82it/s]


In [6]:
predictions

[tensor([[0.0225],
         [0.0273],
         [0.0289]]),
 tensor([[0.0313],
         [0.0329],
         [0.0455]]),
 tensor([[-0.0257],
         [-0.0169],
         [-0.0061]])]

In [7]:
torch.concat(predictions, axis=1).mean(axis=1, keepdim=True)

tensor([[0.0094],
        [0.0144],
        [0.0228]])